In [0]:
import numpy as np


In [1]:
# 1) 
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
b = np.array([1, 2, 3])
c = 5


def func(x):
    x_transpose = np.transpose(x)
    return 1 / 2 * (x @ A @ x_transpose) - b @ x_transpose + c


def f_grad(x):
    return A @ x - b


lambdas = list(np.linalg.eig(A)[0])


NameError: name 'np' is not defined

In [148]:
def project(x):
    return np.array([max(min(x_i, 1), 0) for x_i in x])


def projected_grad():
    alpha = 2.0 / (sum(lambdas))
    traj_opt_step = []
    n = len(b)
    x_start = np.array([1 for _ in range(n)])
    traj_opt_step.append(x_start.copy())
    cur_x = x_start.copy()
    for i in range(200):
        cur_x = cur_x - alpha * f_grad(cur_x)
        cur_x = project(cur_x)
        traj_opt_step.append(cur_x.copy())
    return traj_opt_step


print(projected_grad())


[array([1, 1, 1]), array([0.        , 0.06278172, 0.06278172]), array([0.43918905, 0.        , 0.        ]), array([0.08456618, 0.41161601, 0.205808  ]), array([0.4911049, 0.       , 0.       ]), array([0.03916906, 0.46027249, 0.23013624]), array([0.54220222, 0.        , 0.        ]), array([0.        , 0.50816184, 0.25408092]), array([0.58767378, 0.        , 0.        ]), array([0.        , 0.55077861, 0.2753893 ]), array([0.59765909, 0.        , 0.        ]), array([0.        , 0.56013702, 0.28006851]), array([0.5998518, 0.       , 0.       ]), array([0.        , 0.56219208, 0.28109604]), array([0.60033331, 0.        , 0.        ]), array([0.        , 0.56264335, 0.28132168]), array([0.60043905, 0.        , 0.        ]), array([0.        , 0.56274245, 0.28137123]), array([0.60046227, 0.        , 0.        ]), array([0.        , 0.56276421, 0.28138211]), array([0.60046737, 0.        , 0.        ]), array([0.        , 0.56276899, 0.2813845 ]), array([0.60046849, 0.        , 0.        ]

In [138]:
# 2) 1
from scipy.optimize import linprog


def get_interior_point(A, b):
    n = len(A[0])
    bounds_for_coordinates = [(None, None) for _ in range(n)]
    coefficients_x_i = [0 for _ in range(n)]

    def get_x_i_bounds(i):
        bounds = None
        coefficients_x_i[i] = 1
        min_bound = linprog(coefficients_x_i,
                            A_ub=A,
                            b_ub=b,
                            bounds=bounds_for_coordinates)
        coefficients_x_i[i] = -1
        max_bound = linprog(coefficients_x_i,
                            A_ub=A,
                            b_ub=b,
                            bounds=bounds_for_coordinates)
        coefficients_x_i[i] = 0
        min_x_i = min_bound.fun
        max_x_i = -max_bound.fun
        if min_bound.status == 0 and max_bound.status == 0:
            bounds = [min_x_i, max_x_i]
        elif min_bound.status == 0:
            bounds = [min_x_i, min_x_i + 2]
        elif max_bound.status == 0:
            bounds = [max_x_i - 2, max_x_i]
        elif min_bound.status == 3 and max_bound.status == 3:
            bounds = [-8, 8]
        return bounds

    inner_point = []
    for i in range(n):
        bounds = get_x_i_bounds(i)
        if bounds is None or bounds[0] == bounds[1]:
            return None
        bounds_for_coordinates[i] = (bounds[0], bounds[1])
        inner_point.append((bounds[0] + bounds[1])/2)
    return inner_point


[ 1.55e-06 -2.70e-07]


In [ ]:
# 2) 2
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
b = np.array([1, 2, 3])
c = 5
EqA = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1], [-1, 0, 0, ], [0, -1, 0], [0, 0, -1]])
EqB = np.array([[1], [1], [1], [0], [0], [0]])
m = len(EqB)


def grad_Fi(x):
    grad = 0
    for i in range(m):
        grad += 1.0 / (EqB[i] - EqA[i] @ x) * EqA[i]
    return np.transpose(grad)


def hessian_Fi(x):
    d = np.diag(np.array([1.0 / (EqB[i][0] - EqA[i] @ x) ** 2 for i in range(m)]))
    return EqA.T @ d @ EqA


def grad_f(x):
    return A @ x - b


def hessian_f(x):
    return A


def grad_F(x):
    return lambda t: t * grad_f(x) + grad_Fi(x)


def hessian_F(x):
    return lambda t: t * hessian_f(x) + hessian_Fi(x)


def interior_point_method(x_start):
    cur_x = x_start.copy()
    t = 1.5
    alpha = 1.5
    trajectory = []
    for i in range(50):
        cur_x = cur_x - np.linalg.inv(hessian_F(cur_x)(t)) @ grad_F(cur_x)(t)
        trajectory.append(cur_x.copy())
        t = alpha * t
    return trajectory


interior_point = get_interior_point(EqA, EqB)
print(interior_point)
interior_point_method(interior_point)
